In [83]:
import openai
import pandas as pd


In [84]:
client = openai.Client()


In [85]:
file = client.files.create(
    file=open("sales_data.csv", "rb"),
    purpose="assistants"
)


In [86]:
print(file.id)


file-Po5Fs5df2nRyB7LEMYdtAd


In [87]:
assistant = client.beta.assistants.create(
    name="Analista de Dados",
    instructions="Você é um analista que analisa dados sobre vendas",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4o"
)


In [88]:
#pergunta = "Qual o rating médio das vendas do supermercado"
pergunta = "Gere um gráfico de pizza com o percentual de vendas por linha de produto"


In [89]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)


In [90]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)


In [91]:
# Aguarda a thread rodar
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )


In [92]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")


SyncCursorPage[Message](data=[Message(id='msg_kLqztrphX8UvOzOd8ju68YjU', assistant_id='asst_KqyDw50tqrWb2457gO520Ydg', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-NxeGupFnShoQXjAe18uGb5', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico de pizza mostrando o percentual de vendas por linha de produto. Cada fatia representa o percentual de vendas para uma linha de produto específica em relação ao total de vendas. Se precisar de mais alguma informação ou análise, por favor, me avise!'), type='text')], created_at=1745282168, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Ub5SnlcDtkfOKChvjw1ihymW', status=None, thread_id='thread_7AJucHWdVCNZaFyAqVv6J5KZ'), Message(id='msg_AWpBLCBe6SNi1TnKN8Y2Yu3g', assistant_id='asst_KqyDw50tqrWb2457gO520Ydg', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(ann

In [93]:
#mensagens.data[0].content[0].text.value


In [94]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)


In [95]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem {file_id} salva")
        
        # print(message.content[0].text.value)



====Step message_creation
Para gerar um gráfico de pizza com o percentual de vendas por linha de produto, primeiro precisamos examinar o conteúdo do arquivo enviado para entender sua estrutura. Vou fazer isso agora.

====Step tool_calls
import pandas as pd

# Carregar o arquivo e exibir as primeiras linhas para entender sua estrutura
file_path = '/mnt/data/file-Po5Fs5df2nRyB7LEMYdtAd'
data = pd.read_csv(file_path)
data.head()

====Step message_creation
O arquivo contém dados de transações de vendas, incluindo uma coluna chamada "Product line", que representa as diferentes linhas de produtos, e uma coluna "Total", que representa o valor total das vendas para cada transação. Agora podemos calcular o percentual de vendas por linha de produto e criar um gráfico de pizza com esses dados. Vamos prosseguir com isso.

====Step tool_calls
import matplotlib.pyplot as plt

# Calcular o total de vendas por linha de produto
sales_by_product_line = data.groupby('Product line')['Total'].sum()

# Cal